### Enexis Kleinverbruikgegevens dataset
https://www.enexis.nl/over-ons/wat-bieden-we/andere-diensten/open-data 


Beschrijving:

https://www.enexis.nl/over-ons/-/media/documenten/open-data/toelichting-kleinverbruiksgegevens-open-data-enexis-netbeheer.pdf?modified=00010101000000&la=nl-nl&hash=1A66620D8DBA0D55A55767B507C535BC218B0B14

We hadden al de csv files per jaar vanaf 2010 maar er is nu ook een nieuwe csv file voor 2021 beschikbaar. op de website van Enexis staan alleen de laatste 3 jaar.

Belangrijkste attributen in de dataset:
- POSTCODE_VAN en POSTCODE_TOT, PC6
- PRODUCTSOORT, zowel Gas als Elektriciteit. Filter toepassen op alleen Elektriciteit
- AANSLUITINGEN_AANTAL, aantal aansluitingen in het postcode gebied
- LEVERINGSRICHTING_PERC, Percentage van de aansluitingen dat netto elektriciteits- of gasverbruik heeft. Dit percentage wordt lager naarmate er meer teruglevering plaatsvindt (b.v. vanwege zonnepanelen) 
- SOORT_AANSLUITING  de meest voorkomende ‘Soort aansluiting’, Opties voor elektriciteit: (”#zekeringen x # ampère”) 1x25, 1x35, 1x50, 3x25, 3x35, 3x50, 3x63, 3x80
- SJV_GEMIDDELD, Het gemiddeld standaardjaarverbruik (SJV) in kWh. Het verwachte jaarverbruik van een afnemer op een netaansluiting bij gestandaardiseerde condities en op basis van een genormaliseerd jaar. Bij teruglevering dan is het SJV gesaldeerd.

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import zipfile
import seaborn as sns

In [2]:
# show all columns in the dataframe
pd.set_option('max_columns', None)

In [4]:
# variables used in script
# path to Data folder
DATA = '../data/raw/Enexis_kleinverbruikgegevens'

# Ga naar de folder waarin de data staat
if 'Enexis_kleinverbruikgegevens' not in os.getcwd():
    os.chdir(DATA)

In [5]:
df = pd.DataFrame()
for j in glob.glob('Enexis_kleinverbruiksgegevens*.zip'):
    print(j)
    appenddata = pd.read_csv(j, sep=';', thousands='.', decimal=',',  encoding= 'unicode_escape')
    # jaar kolom toevoegen op basis van de file name
    appenddata['JAAR'] = j[-8:-4]
    df = df.append(appenddata,ignore_index=True)

Enexis_kleinverbruiksgegevens_01012010.zip
Enexis_kleinverbruiksgegevens_01012011.zip
Enexis_kleinverbruiksgegevens_01012012.zip
Enexis_kleinverbruiksgegevens_01012013.zip
Enexis_kleinverbruiksgegevens_01012014.zip
Enexis_kleinverbruiksgegevens_01012015.zip
Enexis_kleinverbruiksgegevens_01012016.zip
Enexis_kleinverbruiksgegevens_01012017.zip
Enexis_kleinverbruiksgegevens_01012018.zip
Enexis_kleinverbruiksgegevens_01012019.zip
Enexis_kleinverbruiksgegevens_01012020.zip
Enexis_kleinverbruiksgegevens_01012021.zip


In [6]:
df.columns

Index(['NETBEHEERDER', 'NETGEBIED', 'STRAATNAAM', 'POSTCODE_VAN',
       'POSTCODE_TOT', 'WOONPLAATS', 'LANDCODE', 'PRODUCTSOORT',
       'VERBRUIKSSEGMENT', 'LEVERINGSRICHTING_PERC', 'AANSLUITINGEN_AANTAL',
       'FYSIEKE_STATUS_PERC', 'SOORT_AANSLUITING_PERC', 'SOORT_AANSLUITING',
       'SJV_GEMIDDELD', 'SJV_LAAG_TARIEF_PERC', 'SLIMME_METER_PERC', 'JAAR'],
      dtype='object')

In [7]:
#df["SOORT_AANSLUITING"].value_counts(dropna=False)

In [8]:
# filter op elektriciteit
df = df[df['PRODUCTSOORT'] == 'ELK']

In [9]:
# check missing values
missing_total = df.isna().sum().sort_values(ascending=False)
print('Number of attributes with Null values: ' + str(len(missing_total[missing_total > 0])))
print(missing_total[missing_total > 0])

Number of attributes with Null values: 3
SOORT_AANSLUITING         107581
SOORT_AANSLUITING_PERC    107512
LEVERINGSRICHTING_PERC       958
dtype: int64


In [10]:
# fill missing values 
df['SOORT_AANSLUITING'] = df['SOORT_AANSLUITING'].fillna('Onbekend')
df['SOORT_AANSLUITING_PERC'] = df['SOORT_AANSLUITING_PERC'].fillna('Onbekend')
df['LEVERINGSRICHTING_PERC'] = df['LEVERINGSRICHTING_PERC'].fillna('Onbekend')

In [11]:
df.head()

,NETBEHEERDER,NETGEBIED,STRAATNAAM,POSTCODE_VAN,POSTCODE_TOT,WOONPLAATS,LANDCODE,PRODUCTSOORT,VERBRUIKSSEGMENT,LEVERINGSRICHTING_PERC,AANSLUITINGEN_AANTAL,FYSIEKE_STATUS_PERC,SOORT_AANSLUITING_PERC,SOORT_AANSLUITING,SJV_GEMIDDELD,SJV_LAAG_TARIEF_PERC,SLIMME_METER_PERC,JAAR
0,Enexis B.V.,ENEXIS,Sasdijk,4251AB,4251AB,WERKENDAM,NL,ELK,KVB,100.0,16,100,Onbekend,Onbekend,4282.0,25.0,0.0,2010
1,Enexis B.V.,ENEXIS,Sasdijk,4251AC,4251AC,WERKENDAM,NL,ELK,KVB,100.0,11,100,Onbekend,Onbekend,5113.0,10.0,0.0,2010
2,Enexis B.V.,ENEXIS,Sasdijk,4251AD,4251AD,WERKENDAM,NL,ELK,KVB,100.0,30,100,Onbekend,Onbekend,4809.0,34.0,0.0,2010
3,Enexis B.V.,ENEXIS,Nieuweweg,4251AE,4251AG,WERKENDAM,NL,ELK,KVB,100.0,21,100,Onbekend,Onbekend,5015.0,44.0,0.0,2010
4,Enexis B.V.,ENEXIS,Koppenhof,4251AH,4251AH,WERKENDAM,NL,ELK,KVB,100.0,12,100,Onbekend,Onbekend,3074.0,22.0,0.0,2010


In [12]:
# voeg attibuut toe met numerieke deel van de postcode
df['PC4'] = df['POSTCODE_VAN'].astype(str).str[0:4]

In [13]:
# extra attribuut toevoegen voor totale standaard jaarverbruik
df['SJV_TOTAAL'] = round(df['AANSLUITINGEN_AANTAL'] * df['SJV_GEMIDDELD'] / 1000) # delen door 1000 om MW ipv KW te krijgen

In [14]:
# group by pc4
df_pc = df.groupby(['PC4','JAAR'], as_index=False)['SJV_TOTAAL'].sum()

In [15]:
df_pc.head()

,PC4,JAAR,SJV_TOTAAL
0,4251,2010,21882.0
1,4251,2011,23084.0
2,4251,2012,24006.0
3,4251,2013,23728.0
4,4251,2014,23623.0


# Dataframe opslaan

In [19]:
os.getcwd()

'C:\\Users\\MarcelKoolwijk\\Documents\\SQL\\jads_enexis\\data\\raw\\Enexis_kleinverbruikgegevens'

In [33]:
# data locatie om de ht5 file op te slaan
data_processed_location = '../../processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

In [35]:
try:
    os.remove('kleinverbruikgegevens_data.h5')
except:
    print ('File nog in gebruik of niet gevonden')

store = pd.HDFStore('kleinverbruikgegevens_data.h5')
store['kleinverbruikgegevens_data'] = df
store.close()

# Meteo data inlezen

In [27]:
df = pd.read_hdf('../processed/meteo_data.h5')

In [28]:
df.columns

Index(['STN', 'YYYYMMDD', 'DDVEC', 'FHVEC', 'FG', 'FHX', 'FHXH', 'FHN', 'FHNH',
       'FXX', 'FXXH', 'TG', 'TN', 'TNH', 'TX', 'TXH', 'T10N', 'T10NH', 'SQ',
       'SP', 'Q', 'DR', 'RH', 'RHX', 'RHXH', 'PG', 'PX', 'PXH', 'PN', 'PNH',
       'VVN', 'VVNH', 'VVX', 'VVXH', 'NG', 'UG', 'UX', 'UXH', 'UN', 'UNH',
       'EV24', 'LOC', 'LAT', 'LNG'],
      dtype='object')

In [29]:
# weerstation van Volkel selecteren voor den Bosch
df = df[df['LOC'] == 'Volkel']

In [30]:
df['LOC'].value_counts()

Volkel    25565
Name: LOC, dtype: int64

In [31]:
df[['YYYYMMDD','TX','LOC']]

,YYYYMMDD,TX,LOC
0,19510131,,Volkel
1,19510201,,Volkel
2,19510202,,Volkel
3,19510203,,Volkel
4,19510204,,Volkel
...,...,...,...
25560,20210123,54,Volkel
25561,20210124,53,Volkel
25562,20210125,51,Volkel
25563,20210126,63,Volkel


In [32]:
df['YYYYMMDD'] = df['YYYYMMDD']